# Invoke Bedrock model for text generation using zero-shot prompt

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use a LLM to generate an email response to a customer who provided negative feedback on the quality of customer service that they received from the support engineer. 

We will use Bedrock's Amazon Titan Text large model using the Boto3 API. 

The prompt used in this example is called a zero-shot prompt because we are not providing any examples of text alongside their classification other than the prompt.

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
To demonstrate the text generation capability of Amazon Bedrock, we will explore the use of Boto3 client to communicate with Amazon Bedrock Converse API. We will demonstrate different configurations available as well as how simple input can lead to desired outputs.

#### Pattern
We will simply provide the Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./imgs/bedrock.jpg)

#### Use case
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, in this notebook we will show how to generate an email with a thank you note based on the customer's previous email.We will use the Amazon Titan Text Large model using the Amazon Bedrock API with Boto3 client. 

## Setup

In [2]:
import json
import os
import sys

import boto3
import botocore


modelId = "amazon.titan-tg1-large"
region = 'us-east-1'

boto3_bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
    )
#- use this for with profile

## Generate text

In [3]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer
"""


Let's start by using the Amazon Titan Large model. The Amazon Titan family of models support a large context window of up to 32k tokens and accepts the following parameters:
- `messages`: Prompt to the LLM
- `inference_config`: These are the parameters that model will take into account while generating the output.

In [4]:
# Base inference parameters.
inference_config = {
        "temperature": 0.1,
        "maxTokens": 4096,
        "topP": 0.95,
}


messages = [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ]

The Amazon Bedrock Converse API provides a consistent interface that works with all models that support messages. This allows you to write code once and use it with different models with an API .`converse`  accepts the following parameter in this example:
- `modelId`: This is the model ARN for the various foundation models available under Amazon Bedrock
- `inferenceConfig`: Inference parameters to pass to the model. Converse supports a base set of inference parameters.
- `messages`: A message consisting of the prompt 

Check [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) for Available text generation model Ids

#### Invoke the Amazon Titan Text language model

First, we explore how the model generates an output based on the prompt created earlier.

##### Complete Output Generation

In [5]:

# Send the message.
try:
    response = boto3_bedrock.converse(
            modelId=modelId,
            messages=messages,
            inferenceConfig=inference_config,
    )
    outputText = response['output']['message']['content'][0]['text']
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

In [6]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)


Customer Service Manager
{EMAIL-1}
John Doe
{EMAIL-2}

Subject: Apology for the Negative Experience

Dear John Doe,

I am writing to express my deepest apologies for the negative experience you had with our customer support engineer. We take customer feedback seriously and are committed to resolving this issue promptly and effectively.

I understand that the service provided did not meet your expectations, and I sincerely apologize for any inconvenience or frustration this may have caused you. We strive to provide exceptional service to all our customers, and it is disappointing to hear that we have fallen short of your expectations.

To address this issue, I have personally reviewed your feedback with our customer support team and taken steps to ensure that this does not happen again. We have implemented additional training for our engineers to ensure that they are fully equipped to handle customer inquiries and provide the highest level of service.

In addition, I would like to offer

##### Streaming Output Generation
Above is an example email generated by the Amazon Titan Large model by understanding the input request and using its inherent understanding of the different modalities. This request to the API is synchronous and waits for the entire output to be generated by the model.

Bedrock also supports that the output can be streamed as it is generated by the model in form of chunks. Below is an example of invoking the model with streaming option. `converse_stream` returns a `EventStream` which you can read from.

_You may want to enable scrolling on your output cell below:_ 

In [7]:
output = []
try:
    response = boto3_bedrock.converse_stream(
            modelId=modelId,
            messages=messages,
            inferenceConfig=inference_config,
    )
    stream = response['stream']
    
    i = 1
    if stream:
        for event in stream:
            if 'contentBlockDelta' in event:
                streaming_text = event['contentBlockDelta']['delta']['text']
                output.append(event['contentBlockDelta']['delta']['text'])
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{streaming_text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Chunk 1**
Bob:
John Doe,

I am writing to express my deepest apologies for the negative experience you had with our customer support engineer.

We take customer feedback very seriously and I want to assure you that we are taking steps to prevent 

		**Chunk 2**
this from happening again. Our customer support engineer has been reprimanded for their behavior and we have implemented additional training to ensure that our customers receive the best possible service in the future.

In addition, we would like to offer you a complimentary month of our service as a token of our apology. Please let me know if y

		**Chunk 3**
ou would like to accept this offer.

Thank you for bringing this issue to our attention and giving us the opportunity to make things right.

Sincerely,
Bob
Customer Service Manager



The above helps to quickly get output of the model and let the service complete it as you read. This assists in use-cases where there are longer pieces of text that you request the model to generate. You can later combine all the chunks generated to form the complete output and use it for your use-case

## Conclusion
You have now experimented with using `boto3` SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use case of generating an email responding to a customer due to their negative feedback.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Anthropic Claude and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You